In [3]:
from dipy.reconst import mapmri
from dipy.data import fetch_cfin_multib, read_cfin_dwi, get_sphere
from dipy.core.gradients import gradient_table
from dipy.io import read_bvals_bvecs

import matplotlib.pyplot as plt
import numpy as np
import nibabel as nib
import os

from mpl_toolkits.axes_grid1 import make_axes_locatable

In [4]:
dwi_big_delta = 0.0334 # scanner default
dwi_small_delta = 0.0322 # scanner default

#dwi_big_delta = 0.045  # desired


In [5]:
data_dir = "/user/pfilipia/home/inria/chu_nice_inria/patients_dmri"

def get_patient_dir(patient_id):
    return os.path.join(
        data_dir, "patient%02d/bids/sub-patient%02d/ses-presurgical" % (patient_id, patient_id)
    )

def get_dwi_file(patient_id):
    return os.path.join(
        get_patient_dir(patient_id), "proc/sub-patient%02d_ses-presurgical_dwi.nii.gz" % patient_id
    )

def get_bval_file(patient_id):
    return os.path.join(
        get_patient_dir(patient_id), "proc/sub-patient%02d_ses-presurgical_dwi.bval" % patient_id
    )

def get_bvec_file(patient_id):
    return os.path.join(
        get_patient_dir(patient_id), "proc/sub-patient%02d_ses-presurgical_dwi.bvec" % patient_id
    )

def get_mask_file(patient_id):
    return os.path.join(
        get_patient_dir(patient_id), "connectivity/tck_streamlines/roi_mask.nii.gz"
    )

def get_stimulation_site_mask_file(patient_id, stimulation_site):
    return os.path.join(
        get_patient_dir(patient_id), "connectivity/streamlines/seed_mask_s%02d_in00.nii" % stimulation_site
    )

def get_microstructure_indices_dir(patient_id):
    indices_dir = os.path.join(
        get_patient_dir(patient_id), "connectivity/tck_streamlines/indices"
    )
    if not os.path.isdir(indices_dir):
        os.mkdir(indices_dir)
    return indices_dir

def get_microstructure_index_file(patient_id, index_name):
    return os.path.join(
        get_microstructure_indices_dir(patient_id), "patient%02d_roi_%s.nii.gz" % (patient_id, index_name)
    )

def get_stimulation_site_microstructure_index_file(patient_id, stimulation_site, index_name):
    return os.path.join(
        get_microstructure_indices_dir(patient_id), 
        "patient%02d_s%02d_%s.nii.gz" % (patient_id, stimulation_site, index_name)
    )


In [6]:
def compute_indices(patient_id):

    dwi_img = nib.load(get_dwi_file(patient_id))
    dwi_data = dwi_img.get_data()

    mask_img = nib.load(get_mask_file(patient_id))
    mask_data = mask_img.get_data()

    bvals, bvecs = read_bvals_bvecs(get_bval_file(patient_id), get_bvec_file(patient_id))
    dwi_gtab = gradient_table(bvals, bvecs, big_delta=dwi_big_delta, small_delta=dwi_small_delta)

    radial_order = 6
    map_model_laplacian_aniso = mapmri.MapmriModel(
        dwi_gtab, radial_order=radial_order,
        laplacian_regularization=True, laplacian_weighting=.2,
        bval_threshold=2000
    )

    mapfit_laplacian_aniso = map_model_laplacian_aniso.fit(dwi_data, mask=mask_data)

    microstructure_indices = {
        'rtop'   : np.array(mapfit_laplacian_aniso.rtop(), dtype=float),
        'rtap'   : np.array(mapfit_laplacian_aniso.rtap(), dtype=float),
        'rtpp'   : np.array(mapfit_laplacian_aniso.rtpp(), dtype=float),
        'msd'    : np.array(mapfit_laplacian_aniso.msd(), dtype=float),
        'qiv'    : np.array(mapfit_laplacian_aniso.qiv(), dtype=float),
        'ng'     : np.array(mapfit_laplacian_aniso.ng(), dtype=float),
        'ng_perp': np.array(mapfit_laplacian_aniso.ng_perpendicular(), dtype=float),
        'ng_par' : np.array(mapfit_laplacian_aniso.ng_parallel(), dtype=float)
    }

    for idx in microstructure_indices:
        idx_img = nib.Nifti1Image(microstructure_indices[idx], dwi_img.affine, dwi_img.header)
        nib.save(idx_img, get_microstructure_index_file(patient_id, idx))


In [7]:
def compute_stimulation_site_indices(patient_id, stimulation_site):

    dwi_img = nib.load(get_dwi_file(patient_id))
    dwi_data = dwi_img.get_data()

    mask_img = nib.load(get_stimulation_site_mask_file(patient_id, stimulation_site))
    mask_data = mask_img.get_data()

    bvals, bvecs = read_bvals_bvecs(get_bval_file(patient_id), get_bvec_file(patient_id))
    dwi_gtab = gradient_table(bvals, bvecs, big_delta=dwi_big_delta, small_delta=dwi_small_delta)

    radial_order = 6
    map_model_laplacian_aniso = mapmri.MapmriModel(
        dwi_gtab, radial_order=radial_order,
        laplacian_regularization=True, laplacian_weighting=.2,
        bval_threshold=2000
    )

    mapfit_laplacian_aniso = map_model_laplacian_aniso.fit(dwi_data, mask=mask_data)

    microstructure_indices = {
        'rtop'   : np.array(mapfit_laplacian_aniso.rtop(), dtype=float),
        'rtap'   : np.array(mapfit_laplacian_aniso.rtap(), dtype=float),
        'rtpp'   : np.array(mapfit_laplacian_aniso.rtpp(), dtype=float),
        'msd'    : np.array(mapfit_laplacian_aniso.msd(), dtype=float),
        'qiv'    : np.array(mapfit_laplacian_aniso.qiv(), dtype=float),
        'ng'     : np.array(mapfit_laplacian_aniso.ng(), dtype=float),
        'ng_perp': np.array(mapfit_laplacian_aniso.ng_perpendicular(), dtype=float),
        'ng_par' : np.array(mapfit_laplacian_aniso.ng_parallel(), dtype=float)
    }

    for idx in microstructure_indices:
        idx_img = nib.Nifti1Image(microstructure_indices[idx], dwi_img.affine, dwi_img.header)
        nib.save(idx_img, get_stimulation_site_microstructure_index_file(patient_id, stimulation_site, idx))


In [ ]:
for patient_id in [4, 7, 10, 11, 12]:
    print("PATIENT_ID:", patient_id)
    for stimulation_site in [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]:
        print("STIMULATION_SITE:", stimulation_site)
        try:
            compute_stimulation_site_indices(patient_id, stimulation_site)
        except:
            print("Exception.")

PATIENT_ID: 4
STIMULATION_SITE: 0
STIMULATION_SITE: 1
STIMULATION_SITE: 2
STIMULATION_SITE: 3
STIMULATION_SITE: 4
STIMULATION_SITE: 5
STIMULATION_SITE: 6
STIMULATION_SITE: 7
STIMULATION_SITE: 8
STIMULATION_SITE: 9
STIMULATION_SITE: 10
STIMULATION_SITE: 11
STIMULATION_SITE: 12
STIMULATION_SITE: 13
STIMULATION_SITE: 14
Exception.
PATIENT_ID: 7
STIMULATION_SITE: 0


In [4]:
dwi_img = nib.load(get_dwi_file(patient_id))
dwi_data = dwi_img.get_data()


In [5]:
mask_img = nib.load(get_mask_file(patient_id))
mask_data = mask_img.get_data()


In [6]:
bvals, bvecs = read_bvals_bvecs(get_bval_file(patient_id), get_bvec_file(patient_id))
dwi_gtab = gradient_table(bvals, bvecs, big_delta=dwi_big_delta, small_delta=dwi_small_delta)


In [ ]:
radial_order = 6
map_model_laplacian_aniso = mapmri.MapmriModel(
    dwi_gtab, radial_order=radial_order,
    laplacian_regularization=True, laplacian_weighting=.2,
    bval_threshold=2000
)

In [ ]:
mapfit_laplacian_aniso = map_model_laplacian_aniso.fit(dwi_data, mask=mask_data)

In [ ]:
microstructure_indices = {
    'rtop'   : np.array(mapfit_laplacian_aniso.rtop(), dtype=float),
    'rtap'   : np.array(mapfit_laplacian_aniso.rtap(), dtype=float),
    'rtpp'   : np.array(mapfit_laplacian_aniso.rtpp(), dtype=float),
    'msd'    : np.array(mapfit_laplacian_aniso.msd(), dtype=float),
    'qiv'    : np.array(mapfit_laplacian_aniso.qiv(), dtype=float),
    'ng'     : np.array(mapfit_laplacian_aniso.ng(), dtype=float),
    'ng_perp': np.array(mapfit_laplacian_aniso.ng_perpendicular(), dtype=float),
    'ng_par' : np.array(mapfit_laplacian_aniso.ng_parallel(), dtype=float)
}

In [ ]:
for idx in microstructure_indices:
    idx_img = nib.Nifti1Image(microstructure_indices[idx], dwi_img.affine, dwi_img.header)
    nib.save(idx_img, get_microstructure_index_file(patient_id, idx))
